In [1]:
pip install transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 31.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/opt/anaconda3/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
texts = ["The movie was great", "The movie was not that great"]
labels = [0, 1]  # Assuming a binary classification task

inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

In [5]:
from torch.utils.data import DataLoader
from transformers import AdamW

# Prepare DataLoader
dataset = list(zip(inputs["input_ids"], inputs["attention_mask"], labels))
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# Optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # Choose appropriate number of epochs
    for batch in loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

/opt/anaconda3/lib/python3.12/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
new_texts = ["My name is Positive"]
inputs = tokenizer(new_texts, padding=True, truncation=True, return_tensors="pt")
logits = model(**inputs).logits
predictions = logits.argmax(dim=1)`
print(predictions)

tensor([1])


In [27]:
#Validating the model

In [33]:
import torch
from sklearn.metrics import accuracy_score

# Assume you have a validation dataset similar to your training data
validation_texts = ["Its a good day", "Its a bad day"]
validation_labels = [0, 1]  # True labels for validation data

# Tokenize validation data
validation_inputs = tokenizer(validation_texts, padding=True, truncation=True, return_tensors="pt")

# Prepare validation DataLoader
validation_dataset = list(zip(validation_inputs["input_ids"], validation_inputs["attention_mask"], validation_labels))
validation_loader = DataLoader(validation_dataset, batch_size=2, shuffle=False)

for epoch in range(5):  # Choose appropriate number of epochs
    model.train()  # Set the model to training mode
    for batch in loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    
    # Validation step
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in validation_loader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = logits.argmax(dim=1)
            
            all_preds.extend(predictions.numpy())  # Collect predictions
            all_labels.extend(labels.numpy())  # Collect true labels
    
    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f'Epoch {epoch + 1}, Validation Accuracy: {accuracy:.4f}')

Epoch 1, Validation Accuracy: 0.5000
Epoch 2, Validation Accuracy: 0.5000
Epoch 3, Validation Accuracy: 0.5000
Epoch 4, Validation Accuracy: 0.5000
Epoch 5, Validation Accuracy: 0.5000
